In [1]:
import pandas as pd

import anndata
import scanpy as sc
from ALLCools.mcds import MCDS
from ALLCools.clustering import log_scale


In [2]:
var_dim_er ='geneslop2k'
chrom_to_remove = ['chrX', 'chrY', 'chrM', 'chrL']
excluded_L1_annot = ['MGC', 'ODC', 'ASC', 'OPC', 'CB', 'CBX']
mc_type='CHN'


In [3]:
region_to_subregion = {'CTX': ['MOp','SSp','ACA','AI','RSP','AUD','PTLp','VIS'],
                       'HIP': ['CAa','CAp','DGa','DGp'],
                       'RHP': ['ENT'],
                       'OLF': ['MOB'],
                       'PIR': ['PIRa','PIRp'],
                       'STR': ['STR'],
                       'PAL': ['PAL'],
                       'AMY': ['AMY'],
                       'TH': ['THm','THl','THp'],
                       'HY': ['HY'],
                       'MB': ['SC','MRN','VTA','PAG','IC'],
                       'HB': ['P','MY']
                      }
rs2_subregion_dict = {
    'MOp': ['3C', '4B'],
    'SSp': ['6B', '7B'],
    'ACA': ['3A', '4A', '5A'],
    'AI': ['3D'],
    'RSP': ['9A', '10A'],
    'AUD': ['9D', '10C'],
    'PTLp': ['9B'],
    'VIS': ['11B', '12B'],
    'ENT': ['10D11D12D13B'],
    'CAa': ['8E9H'],
    'CAp': ['10E11E'],
    'DGa': ['8J9J'],
    'DGp': ['10F11F'],
    'PIRa': ['2D3E4F5G'],
    'PIRp': ['6D7D8D9E'],
    'MOB': ['1C', '2E'],
    'PAL': ['4H5H6F7F', '6F7F'],
    'STR': ['4D5E6E'],
    'AMY': ['7H8H9G'],
    'THl': ['7E8F'], 
    'THm': ['7G8G'], 
    'THp': ['9K10G'],
    'HY': ['6H7J8K9L'],
    'SC': ['11G12F13C'],
    'MRN': ['10H11H12H'],
    'VTA': ['10J11J'],
    'PAG': ['12G13D'],
    'IC': ['14A'],
    'P': ['12J13E13F14C14D'],
    'MY': ['15C16C17B18B']
}

In [4]:
group_name = 'STR'

In [5]:
t = group_name
selected_ER_slice = [x for xx in region_to_subregion[t] for x in rs2_subregion_dict[xx]]


In [6]:
meta = pd.read_csv('/cemba/CEMBA_RS2/cell_48032_RS2_meta_nooutlier.csv.gz', index_col=0, header=0)
meta


,mCCCFrac,mCGFrac,mCHFrac,FinalmCReads,Plate,Exp,Source,Slice,Target,Gender,L1,L2,L3,L4,L1_annot,PlateNormReads
cell,,,,,,,,,,,,,,,,
180118_CEMBA_mm_P56_P63_RS2_180117_P1_180117_P2_A10_AD001,0.006500,0.775850,0.032027,1090870.0,180117_P1,Tm3C,MOp,3C,SC,male,c14,c14_c6,c14_c6_c0,c14_c6_c0_c0,ET,-0.555116
180118_CEMBA_mm_P56_P63_RS2_180117_P1_180117_P2_A10_AD002,0.006709,0.775146,0.034091,1458065.0,180117_P1,Tf3C,MOp,3C,SC,female,c14,c14_c5,c14_c5_c0,c14_c5_c0_c0,ET,-0.136540
180118_CEMBA_mm_P56_P63_RS2_180117_P1_180117_P2_A10_AD004,0.007131,0.775895,0.035900,1614993.0,180117_P1,Tm4B,MOp,4B,SC,male,c14,c14_c6,c14_c6_c1,c14_c6_c1_c0,ET,0.010933
180118_CEMBA_mm_P56_P63_RS2_180117_P1_180117_P2_A10_AD006,0.006032,0.773345,0.028591,1598778.0,180117_P1,Tf4B,MOp,4B,SC,female,c14,c14_c0,c14_c0_c0,c14_c0_c0_c1,ET,-0.003626
180118_CEMBA_mm_P56_P63_RS2_180117_P1_180117_P2_A10_AD007,0.005007,0.773471,0.022022,762007.0,180117_P2,Pm3C,MOp,3C,STR,male,c1,c1_c1,c1_c1_c3,c1_c1_c3_c2,IT-Sup,-1.169393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pool180_Plate9-6-M14-O24,0.004759,0.737593,0.018632,1237195.0,Pool180_Plate9,Zf10D11D12D13B,ENT,10D11D12D13B,PFC,female,c4,c4_c3,c4_c3_c1,c4_c3_c1_c2,OLF,0.305202
Pool180_Plate9-6-M14-P11,0.005464,0.759072,0.024842,1103912.0,Pool180_Plate9,Zm10D11D12D13B,ENT,10D11D12D13B,PFC,male,c4,c4_c4,c4_c4_c2,c4_c4_c2_c0,OLF,0.140754
Pool180_Plate9-6-M14-P12,0.006196,0.792529,0.029565,818886.0,Pool180_Plate9,Zm10D11D12D13B,ENT,10D11D12D13B,PFC,male,c4,c4_c6,c4_c6_c1,c4_c6_c1_c0,OLF,-0.290136


In [7]:
selc = meta.index[meta['Slice'].isin(selected_ER_slice) & ~meta['L1_annot'].isin(excluded_L1_annot)]
print(len(selc))


198


In [8]:
mcds = MCDS.open('/cemba/CEMBA_RS2/CEMBA.epiretro.mcds', var_dim=var_dim_er, use_obs=selc)
mcds


<xarray.MCDS>
Dimensions:             (cell: 198, count_type: 2, geneslop2k: 32208, mc_type: 2)
Coordinates:
  * cell                (cell) <U99 'Pool129_RS2_Plate10-1-E14-A1' ... 'Pool1...
  * count_type          (count_type) <U3 'mc' 'cov'
  * geneslop2k          (geneslop2k) <U18 'ENSMUSG00000051951' ... 'ENSMUSG00...
    geneslop2k_chrom    (geneslop2k) <U5 'chr1' 'chr1' 'chr1' ... 'chrM' 'chrM'
    geneslop2k_end      (geneslop2k) int64 3673498 3515553 ... 16070 16299
    geneslop2k_start    (geneslop2k) int64 3203901 3464587 ... 11552 12145
  * mc_type             (mc_type) <U3 'CGN' 'CHN'
Data variables:
    geneslop2k_da       (cell, geneslop2k, mc_type, count_type) uint32 dask.array<chunksize=(198, 32208, 1, 1), meta=np.ndarray>
    geneslop2k_da_frac  (cell, geneslop2k, mc_type) float64 dask.array<chunksize=(198, 32208, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  geneslop2k

In [9]:
use_genes = pd.read_csv(f'use_genes.txt', header=None, index_col=0).index
use_genes

Index(['ENSMUSG00000025905', 'ENSMUSG00000033740', 'ENSMUSG00000067879',
       'ENSMUSG00000042501', 'ENSMUSG00000048960', 'ENSMUSG00000016918',
       'ENSMUSG00000025938', 'ENSMUSG00000025935', 'ENSMUSG00000025937',
       'ENSMUSG00000025932',
       ...
       'ENSMUSG00000054843', 'ENSMUSG00000025089', 'ENSMUSG00000025092',
       'ENSMUSG00000041362', 'ENSMUSG00000006270', 'ENSMUSG00000025094',
       'ENSMUSG00000087095', 'ENSMUSG00000043969', 'ENSMUSG00000057858',
       'ENSMUSG00000117790'],
      dtype='object', name=0, length=3402)

In [10]:
hvf_judge = mcds.get_index(mcds.var_dim)
hvf_judge = pd.Series(hvf_judge.isin(use_genes), index=hvf_judge)
hvf_judge

geneslop2k
ENSMUSG00000051951    False
ENSMUSG00000089699    False
ENSMUSG00000102331    False
ENSMUSG00000102343    False
ENSMUSG00000025900    False
                      ...  
ENSMUSG00000065947    False
ENSMUSG00000064363    False
ENSMUSG00000064367    False
ENSMUSG00000064368    False
ENSMUSG00000064370    False
Length: 32208, dtype: bool

In [11]:
mcds.coords['_feature_select'] = hvf_judge
# mcds = mcds.sel({mcds.var_dim:use_genes})
er = mcds.get_adata(mc_type='CHN', select_hvf='_feature_select')
er

AnnData object with n_obs × n_vars = 198 × 3402
    var: 'chrom', 'end', 'start', '_feature_select'

In [12]:
# log_scale use StandardScaler which normalized to unit variance 
log_scale(er, with_mean=True)
er.X = -er.X


In [13]:
er = er[:, use_genes]
er.obs = meta.loc[er.obs.index,:]


In [14]:
er.write_h5ad(f'{t}_{er.shape[0]}_rs2_mch.h5ad')